In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import plotly.express as px
from tabula import read_pdf

In [ ]:
x, y = read_pdf('pdfs/Current_Property_Sales_Data_File_Format_2001_to_Current.pdf', pages=[1,2])

In [ ]:
cols = x[x.columns[0]].dropna().to_list()[21:] + y[y.columns[0]].dropna().to_list()[:-7]
cols = [i.strip().lower().replace('%', '').replace('/', '').strip().replace(".", "").replace('  ', ' ').replace(' ', '_') for i in cols]
len(cols)

In [ ]:
def parse_data_file(file_path):
    # Initialize containers for different types of records
    data = {'HEADER': None, 'SALES': [], 'FOOTER': None}

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(';')
            record_type = parts[0]

            if record_type == 'A':  # Header record
                data['HEADER'] = {
                    'Record Type': parts[0],
                    'District Code': parts[1],
                    'Download Date / Time': parts[2],
                    'Submitter\'s Userid': parts[3],
                }
            elif record_type == 'B':  # Sales record
                data['SALES'].append({
                    'Record Type': parts[0],
                    'District Code': parts[1],
                    'Source': parts[2],
                    'Valuation_num': parts[3],
                    'Property Id.': parts[4],
                    'Unit_num': parts[5],
                    'House_num': parts[6],
                    'Street_name': parts[7],
                    'Suburb_name': parts[8],
                    'Postcode': parts[9],
                    'Contract_date': parts[10],
                    'Purchase_Price': parts[11],
                    'Land_Description': parts[12],
                    'Area': parts[13],
                    'Area_type': parts[14],
                    'Dimensions': parts[15],
                    'Comp_code': parts[16],
                    'Zone_code': parts[17],
                    # 'Vendor_name' and 'Purchaser_name' are removed for privacy reasons
                })
            elif record_type == 'Z':  # Footer record
                data['FOOTER'] = {
                    'Record Type': parts[0],
                    'Total records': parts[1],
                    'Total B records': parts[2],
                }

    return data

In [ ]:
data = [line[:-1].split(';') for line in tqdm(open('data.csv').read().split('\n')) if ';' in line]
df = pd.DataFrame([d for d in tqdm(data) if len(d) == 24], columns=cols)

In [ ]:
x = df.groupby('primary_purpose').size()
x[x > 100].sort_values()

In [ ]:
len(df)

In [ ]:
df2 = df[df.primary_purpose == 'RESIDENCE']
df2 = df2[df2.settlement_date != '']
df2 = df2.replace(r'^\s*$', np.nan, regex=True)
df2 = df2[df2.area_type == 'M']
df2['area'] = df2.area.astype('float64')
df2['purchase_price'] = df2.purchase_price.astype('float64')
df2['ts'] = pd.to_datetime(df2.settlement_date)
df2['ppm'] = df2.purchase_price / df2.area
df2 = df2.sort_values(['ts', 'sale_counter'])

In [ ]:
df2.to_parquet('data.pq')

In [ ]:
df2 = pd.read_parquet('data.pq').reset_index(drop=True)
df2 = df2[df2.settlement_date != '20320516']
df2 = df2.sort_values('ts')
df2 = df2[df2.settlement_date > '20010101']

In [ ]:
# df2.tail().T

In [ ]:
px.scatter(df2.set_index('ts').resample('10D').ppm.mean().dropna())

In [ ]:
df2.groupby(df2.ts.round('10D')).ppm.max().sort_values(ascending=False).astype('int64')

In [ ]:
df2[df2.property_locality.str.contains('ROSE')].head()

In [ ]:
df3[df3[8read_parquetntains('GIBBEN')].head().T

In [ ]:
df4 = df3[df3[15].str.strip() != '']
df4['price'] = df4[15].astype('float64')
df4 = df4[df4.ts > pd.Timestamp('2012-01-01')]
df5 = df4.groupby(df4.ts.round('20D'))['price'].median()

In [ ]:
df3.head()

In [ ]:
px.scatter(df5, log_y=True)

In [ ]:
import pandas as pd

In [ ]:
df

In [ ]:
from tqdm.auto import tqdm

In [ ]:
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor


def parse_1990_file(file_path):
    # Initialize containers for different types of records
    data = {'HEADER': None, 'SALES': [], 'FOOTER': None}

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(';')
            record_type = parts[0]

            if record_type == 'A':  # Header record
                data['HEADER'] = {
                    'Record Type': parts[0],
                    'District Code': parts[1],
                    'Download Date / Time': parts[2],
                    'Submitter\'s Userid': parts[3],
                }
            elif record_type == 'B':  # Sales record
                data['SALES'].append({
                    'Record Type': parts[0],
                    'District Code': parts[1],
                    'Source': parts[2],
                    'Valuation_num': parts[3],
                    'Property Id.': parts[4],
                    'Unit_num': parts[5],
                    'House_num': parts[6],
                    'Street_name': parts[7],
                    'Suburb_name': parts[8],
                    'Postcode': parts[9],
                    'Contract_date': parts[10],
                    'Purchase_Price': parts[11],
                    'Land_Description': parts[12],
                    'Area': parts[13],
                    'Area_type': parts[14],
                    'Dimensions': parts[15],
                    'Comp_code': parts[16],
                    'Zone_code': parts[17],
                    # 'Vendor_name' and 'Purchaser_name' are removed for privacy reasons
                })
            elif record_type == 'Z':  # Footer record
                data['FOOTER'] = {
                    'Record Type': parts[0],
                    'Total records': parts[1],
                    'Total B records': parts[2],
                }

    return data

def parse_sales_data_file(file_path):
    with open(file_path, 'r') as file:
        data = {'HEADER': None, 'FOOTER': None, 'SALES': []}
        sales_index = {}  # To index sales entries by the first 5 columns

        counter = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'Z':0}
        for line in file:
            line = line.strip()
            if line == '' or set(line) == {';'}:
                continue
            parts = line.split(';')
            segments = len(parts)
            record_type = parts[0]
            
            key = tuple(parts[1:5])  # First 5 columns as a tuple to use as a key

            counter[record_type] += 1
            if record_type == 'A':  # Header
                if segments == 5:
                    parts = [parts[0]] + ['NA'] + parts[1:]
                if len(parts) != 6:
                    raise ValueError("Invalid File Header:", line)
                    
                data['HEADER'] = {
                    'File Type': parts[1],
                    'District Code': parts[2],
                    'Download Date Time': parts[3],
                    'Submitter\'s Userid': parts[4]
                }
            elif record_type == 'Z':  # Footer
                data['FOOTER'] = {
                    'Total records': parts[1],
                    'Total B records': parts[2],
                    'Total C records': parts[3],
                    'Total D records': parts[4]
                }
            elif record_type == 'B':  # New sale entry
                area_type = {'M': 'Square Meters', 'H': 'Hectares'}.get(parts[12], parts[12])
                nature_property = {'V': 'Vacant', 'R': 'Residence', '3': 'Other'}.get(parts[17], parts[17])
                sales_index[key] = {
                    'Record Type': parts[0],
                    'District Code': parts[1],
                    'Property Id': parts[2],
                    'Sale Counter': parts[3],
                    'Download Date Time': parts[4],
                    'Property Name': parts[5],
                    'Property Unit Number': parts[6],
                    'Property House Number': parts[7],
                    'Property Street Name': parts[8],
                    'Property Locality': parts[9],
                    'Property Post Code': parts[10],
                    'Area': parts[11],
                    'Area Type': area_type,
                    'Contract Date': parts[13],
                    'Settlement Date': parts[14],
                    'Purchase Price': parts[15],
                    'Zoning': parts[16],
                    'Nature Property': nature_property,
                    'Primary Purpose': parts[18],
                    'Strata Number': parts[19],
                    'Component code': parts[20],
                    'Sale Code': parts[21],
                    '% Interest Sale': parts[22],
                    'Dealing Number': parts[23],
                    'Property Legal Descriptions': [],  # List to hold multiple C records
                    'Purchaser – Vendor': []  # List to hold multiple D records
                }
                data['SALES'].append(sales_index[key])
            elif record_type == 'C' and key in sales_index:
                sales_index[key]['Property Legal Descriptions'].append(parts[5])
            elif record_type == 'D' and key in sales_index:
                purchaser_vendor = {'P': 'Purchaser', 'V': 'Vendor'}.get(parts[5], parts[5])
                sales_index[key]['Purchaser – Vendor'].append(purchaser_vendor)
    
    counter['records'] = sum(counter.values())
    validate = {k.split()[1]: int(v) for k,v in data['FOOTER'].items()}
    # assert all(counter[k] == validate[k] for k in validate), (file_path, counter, validate)
    return data

def handle_path(path):
    try:
        if 'ARCHIVE_SALES' in path.name:
            res = parse_1990_file(path)
            res = 'Archive', res
        elif 'SALES_DATA_NNME' in path.name:
            res = parse_sales_data_file(path)
            res = 'Sales', res
        else:
            res = 'Unknown', path
        return res
    except:
        print(file)
        raise

def load_all_data(base):
    archives = []
    datas = []
    paths = Path(base).glob('*.DAT')
    datas = {
        'Archive': [],
        'Sales': [],
        'Unknown': []
    }
    with ProcessPoolExecutor() as executor:
        for kind, res in executor.map(handle_path, paths):
            datas[kind].append(res)
    return datas


In [ ]:
path = 'test/data/001_SALES_DATA_NNME_01012006.DAT'
# path = 'test/data/276_SALES_DATA_NNME_21032022.DAT'

In [ ]:
load_all_data('test/data/')

In [ ]:
!head test/data/656_SALES_DATA_NNME_03122001.DAT

In [ ]:
!head test/data/656_SALES_DATA_08072019.DAT




In [ ]:
from collections import defaultdict
ctr = defaultdict(lambda: 0)
for f in tqdm(list(Path('test/data/').glob('*.DAT'))):
    for line in open(f):
        if len(line) >= 1:
            fchar = line[0]
            if fchar in {'A', 'Z'}:
                continue
        ctr[(fchar, line.count(';'))] += 1
        

In [ ]:
dict(ctr.items())

In [ ]:
from tabula import read_pdf

tbls = read_pdf('./pdfs/Property_Sales_Data_File_Zone_Codes_and_Descriptions_V2.pdf', pages=[1,2])

In [ ]:
code_to_name = {}

for k, v in tbls[0].to_numpy():
    if k.isupper() and len(k) <= 3:
        if k.startswith('RU'):
            v = v.rsplit(' ', 1)[0]
        
        code_to_name[k] = v

In [ ]:
for i, j, k in tbls[1].to_numpy():
    if isinstance(i, str):
        
        print(i)

In [ ]:
from tabula import read_pdf

tbls = read_pdf('./pdfs/Property_Sales_Data_File_District_Codes_and_Names.pdf', pages=[1,2])

In [ ]:
tbls[0]

In [ ]:
lines = open('districts.txt').read().split('\n')

In [ ]:
CODE_TO_DISTRICT = {}

data = {
    'codes': [],
    'districts': []
}
w = ''

for line in lines:
    if line == 'District Code':
        w = 'codes'
        if len(data[w]) > 0:
            for k,v in zip(data['codes'], data['districts']):
                CODE_TO_DISTRICT[k] = v
        data[w] = []
        continue
    if line == 'Council Name':
        w = 'districts'
        data[w] = []
        continue
    print(w, line)
    data[w].append(line.strip())
        

In [ ]:
CODE_TO_DISTRICT

In [ ]:
from pathlib import Path
Path('./foo').mkdir(parents=True, exist_ok=True)

In [1]:
import pandas as pd
import polars as pl
import plotly.express as px

In [2]:
df = pl.read_csv('land_value.csv', infer_schema_length=1000000)

In [3]:
len(df)

13474031

In [4]:
df.head()

district_code,district_name,property_id,file_datetime,property_name,property_unit_number,property_house_number,property_street_name,property_locality,property_post_code,area,area_type,contract_date,settlement_date,purchase_price,zone_code,zone_name,nature_property,primary_purpose,strata_number,component_code,sale_code,interest_sale,dealing_number,property_description,purchaser_vendor,dimensions,filetype
i64,str,i64,str,str,str,str,str,str,i64,f64,str,i64,i64,i64,str,str,str,str,i64,str,str,i64,str,str,str,str,str
234,"""BROKEN HILL""",2613690,"""20130325 01:38…",null,null,"""714""","""BERYL ST""","""BROKEN HILL""",2880,1328.0,"""Square Meters""",20121128,20130205,280000,"""A""","""Residential""","""Residence""","""RESIDENCE""",null,"""CC""",null,0,"""AH554050""","""3301, 3909/757…","""Purchaser, Pur…",null,"""sales"""
234,"""BROKEN HILL""",2613771,"""20130325 01:38…",null,null,"""102""","""BERYL ST""","""BROKEN HILL""",2880,505.9,"""Square Meters""",20130207,20130307,77500,"""A""","""Residential""","""Residence""","""RESIDENCE""",null,"""CD""",null,0,"""AH615023""","""B/341084""","""Purchaser, Ven…",null,"""sales"""
234,"""BROKEN HILL""",2619331,"""20130325 01:38…",null,null,"""18 A""","""MORGAN ST""","""BROKEN HILL""",2880,1024.0,"""Square Meters""",20130125,20130301,130000,"""A""","""Residential""","""Residence""","""RESIDENCE""",null,"""AM""",null,0,"""AH610946""","""5273/757298""","""Purchaser, Pur…",null,"""sales"""
234,"""BROKEN HILL""",2620431,"""20130325 01:38…",null,null,"""223""","""PELL ST""","""BROKEN HILL""",2880,505.9,"""Square Meters""",20130207,20130307,87500,"""A""","""Residential""","""Residence""","""RESIDENCE""",null,"""AL""",null,0,"""AH615022""","""B/974501""","""Purchaser, Ven…",null,"""sales"""
234,"""BROKEN HILL""",2615579,"""20130325 01:38…",null,null,"""418""","""COBALT ST""","""BROKEN HILL""",2880,499.5,"""Square Meters""",20130206,20130308,92000,"""A""","""Residential""","""Residence""","""RESIDENCE""",null,"""CD""",null,0,"""AH615737""","""['A/331398 A/3…","""['Purchaser', …",null,"""sales"""
